In [9]:
import pandas as pd
import os
import shutil

# Carga los datos de los 1000 estudiantes
url_estudiantes = 'https://raw.githubusercontent.com/TomasUR95/Trabajo-Final-Algoritmia-y-Programaci-n/main/Datasets/Estudiantes.csv'
estudiantes_df = pd.read_csv(url_estudiantes)

# Función para generar el código único de la asignatura
def generar_codigo_asignatura(nombre_asignatura, semestre, creditos, consecutivo):
    codigo_nombre = ''.join([word[0].upper() for word in nombre_asignatura.split() if word]).ljust(3, 'X')[:3]
    codigo_semestre = str(semestre).zfill(2)
    codigo_creditos = str(creditos)
    codigo_consecutivo = str(consecutivo).zfill(2)
    return codigo_nombre + codigo_semestre + codigo_creditos + codigo_consecutivo

# Función para calcular horas de trabajo docente e independiente
def calcular_horas_trabajo(creditos):
    htd = creditos * 32
    hti = creditos * 40
    return htd, hti

# Función para distribuir estudiantes en grupos
def distribuir_estudiantes(estudiantes, cupo):
    n_estudiantes = len(estudiantes)
    grupos = [estudiantes.iloc[i:i+cupo] for i in range(0, n_estudiantes, cupo)]
    return grupos

# Función para generar listas de cursos
def generar_listas_cursos(semestre, asignaturas_semestre, estudiantes_df):
    listas_cursos = []
    estudiantes_por_curso = {}

    # Calcular el número de estudiantes por semestre basado en el porcentaje
    porcentaje_estudiantes = tabla_porcentaje[semestre] / 100
    n_estudiantes_semestre = int(len(estudiantes_df) * porcentaje_estudiantes)
    estudiantes_semestre = estudiantes_df.sample(n=n_estudiantes_semestre, random_state=semestre).reset_index(drop=True)

    for asignatura in asignaturas_semestre[semestre]:
        nombre_asignatura = asignatura['nombre']
        creditos = asignatura['creditos']
        codigo_semestre = str(semestre).zfill(2)

        codigo_asignatura = generar_codigo_asignatura(nombre_asignatura, semestre, creditos, 1)
        htd, hti = calcular_horas_trabajo(creditos)

        # Seleccionar estudiantes para la asignatura
        grupos = distribuir_estudiantes(estudiantes_semestre, cupo_por_aula[semestre])

        estudiantes_por_curso[codigo_asignatura] = {'nombre_asignatura': nombre_asignatura, 'grupos': grupos}

        for i, grupo in enumerate(grupos, start=1):
            codigo_grupo = str(i).zfill(2)
            codigo_curso = f"{codigo_asignatura}-{nombre_asignatura.replace(' ', '').capitalize()}-{len(grupo)}-{codigo_grupo}"
            total_estudiantes = len(grupo)

            lista_curso = {
                'Código Asignatura': codigo_asignatura,
                'Nombre Asignatura': nombre_asignatura,
                'Horas de trabajo docente': htd,
                'Horas de trabajo independiente': hti,
                'Número total de estudiantes': total_estudiantes,
                'Código del grupo': codigo_curso,
                'Total de cursos asignados': len(grupos),
                'Fecha de creación': '20240601'  # Fecha de creación del documento
            }
            listas_cursos.append(lista_curso)

            # Crea directorios si no existen
            directorio = f"Ruta_Trabajo_Final/Semestre_{semestre}/{nombre_asignatura.replace(' ', '_').capitalize()}"
            os.makedirs(directorio, exist_ok=True)

            # Guarda grupo en archivo Excel
            excel_path = f"{directorio}/{codigo_curso}.xlsx"
            grupo.to_excel(excel_path, index=False)

            # Guarda grupo en archivo CSV
            csv_path = f"{directorio}/{codigo_curso}.csv"
            grupo.to_csv(csv_path, index=False)

    return listas_cursos, estudiantes_por_curso

# Función para imprimir listas de cursos
def imprimir_listas(listas_cursos):
    for curso in listas_cursos:
        print("Código Asignatura:", curso['Código Asignatura'])
        print("Nombre Asignatura:", curso['Nombre Asignatura'])
        print("Horas de trabajo docente:", curso['Horas de trabajo docente'])
        print("Horas de trabajo independiente:", curso['Horas de trabajo independiente'])
        print("Número total de estudiantes:", curso['Número total de estudiantes'])
        print("Código del curso:", curso['Código del grupo'])
        print("Total de cursos asignados:", curso['Total de cursos asignados'])
        print("Fecha de creación:", curso['Fecha de creación'])
        print()

# Función para imprimir listas de estudiantes
def imprimir_listas_estudiantes(estudiantes_por_curso):
    for codigo_curso, info in estudiantes_por_curso.items():
        nombre_asignatura = info['nombre_asignatura']
        grupos = info['grupos']
        print("Curso:", codigo_curso)
        print("Nombre Asignatura:", nombre_asignatura)
        print("Estudiantes:")
        for i, grupo in enumerate(grupos, start=1):
            print("Grupo", i, ":", grupo['Estudiantes'].tolist())
        print()

# Datos de porcentaje y cupo por aula
tabla_porcentaje = {
    1: 14,
    2: 13,
    3: 12,
    4: 11,
    5: 10,
    6: 10,
    7: 9,
    8: 8,
    9: 7,
    10: 6
}

cupo_por_aula = {
    1: 30,
    2: 30,
    3: 30,
    4: 25,
    5: 25,
    6: 25,
    7: 20,
    8: 20,
    9: 20,
    10: 10
}

asignaturas_semestre = {
    1: [
        {'nombre': 'Algebra y Trigonometria', 'creditos': 3},
        {'nombre': 'Calculo Diferencial', 'creditos': 3},
        {'nombre': 'Geometria Vectorial y Analitica', 'creditos': 3},
        {'nombre': 'Vivamos la Universidad', 'creditos': 1},
        {'nombre': 'Ingles I', 'creditos': 1},
        {'nombre': 'Lectoescritura', 'creditos': 3},
        {'nombre': 'Introduccion a la Ingenieria Industrial', 'creditos': 2},
    ],
    2: [
        {'nombre': 'Cálculo Integral', 'creditos': 3},
        {'nombre': 'Álgebra Lineal', 'creditos': 3},
        {'nombre': 'Habilidades Gerenciales', 'creditos': 3},
        {'nombre': 'Descubriendo la Física', 'creditos': 3},
        {'nombre': 'Ingles II', 'creditos': 1},
        {'nombre': 'Gestión de las Organizaciones', 'creditos': 3},
    ],
    3: [
        {'nombre': 'Gestión Contable', 'creditos': 3},
        {'nombre': 'Física Mecánica', 'creditos': 3},
        {'nombre': 'Teoría General de Sistemas', 'creditos': 3},
        {'nombre': 'Probabiliadad e inferencia estadistica', 'creditos': 3},
        {'nombre': 'Ingles III', 'creditos': 1},
        {'nombre': 'Algoritmia y Programacion', 'creditos': 3},
    ],
    4: [
        {'nombre': 'Ingenieria economica', 'creditos': 3},
        {'nombre': 'Gestión de Métodos y Tiempos', 'creditos': 4},
        {'nombre': 'Diseño de esperimentos y analisis de regresion', 'creditos': 3},
        {'nombre': 'Optimizacion', 'creditos': 3},
        {'nombre': 'Ingles IV', 'creditos': 1},
        {'nombre': 'Electiva fisica', 'creditos': 3},
    ],
    5: [
        {'nombre': 'Gestion financiera', 'creditos': 3},
        {'nombre': 'Gestion por procesos', 'creditos': 3},
        {'nombre': 'Dinamica de sistemas', 'creditos': 3},
        {'nombre': 'Muestreo y series de tiempo', 'creditos': 3},
        {'nombre': 'Procesos estocasticos y analisis de decision', 'creditos': 3},
        {'nombre': 'Ingles V', 'creditos': 1},
        {'nombre': 'Electiva de laboratorio integrado de fisica', 'creditos': 3},
    ],
    6: [
        {'nombre': 'Formación ciudadana y contitucional', 'creditos': 3},
        {'nombre': 'Normalización y control de calidad', 'creditos': 3},
        {'nombre': 'Gestion tecnologica', 'creditos': 3},
        {'nombre': 'Simulación', 'creditos': 3},
        {'nombre': 'Formilación de proyectos de investigación', 'creditos': 3},
        {'nombre': 'Ingles VI', 'creditos': 1},
        {'nombre': 'Electiva en Humanidades I', 'creditos': 3},
    ],
    7: [
        {'nombre': 'Legislación', 'creditos': 3},
        {'nombre': 'Emprendimiento', 'creditos': 2},
        {'nombre': 'Diseño de Sistemas Productivos', 'creditos': 3},
        {'nombre': 'Énfasis profesional I', 'creditos': 3},
        {'nombre': 'Electiva complementaria I', 'creditos': 3},
        {'nombre': 'Electiva en humanidades II', 'creditos': 3},
    ],
    8: [
        {'nombre': 'Formulación y Evaluación de Proyectos de Inversión', 'creditos': 3},
        {'nombre': 'Administración de la Producción y del Servicio', 'creditos': 3},
        {'nombre': 'Énfasis profesional II', 'creditos': 3},
        {'nombre': 'Énfasis profesional I', 'creditos': 3},
        {'nombre': 'Electiva complementaria II', 'creditos': 3},
        {'nombre': 'Electiva en humanidades III', 'creditos': 3},
    ],
    9: [
        {'nombre': 'Gestión de Proyectos', 'creditos': 3},
        {'nombre': 'Gestión de la Cadena de Abastecimiento', 'creditos': 2},
        {'nombre': 'Ingeniería del Mejoramiento Continuo', 'creditos': 3},
        {'nombre': 'Énfasis profesional III', 'creditos': 3},
        {'nombre': 'Electiva complementaria III', 'creditos': 3},
        {'nombre': 'Electiva en humanidades IV', 'creditos': 3},
    ],
    10: [
        {'nombre': 'Prácticas académicas', 'creditos': 12},
    ],
}

# Genera las listas de cursos para cada semestre e imprimir
for semestre in range(1, 11):
    listas_semestre, estudiantes_por_curso = generar_listas_cursos(semestre, asignaturas_semestre, estudiantes_df)
    print("Listas de cursos para el semestre", semestre)
    imprimir_listas(listas_semestre)

    # Imprime las lista de estudiantes por curso
    print("\nListas de estudiantes por curso para el semestre", semestre)
    imprimir_listas_estudiantes(estudiantes_por_curso)

# Comprime la carpeta 'Ruta_Trabajo_Final' en un archivo ZIP
shutil.make_archive('Ruta_Trabajo_Final', 'zip', 'Ruta_Trabajo_Final')

Listas de cursos para el semestre 1
Código Asignatura: AYT01301
Nombre Asignatura: Algebra y Trigonometria
Horas de trabajo docente: 96
Horas de trabajo independiente: 120
Número total de estudiantes: 30
Código del curso: AYT01301-Algebraytrigonometria-30-01
Total de cursos asignados: 5
Fecha de creación: 20240601

Código Asignatura: AYT01301
Nombre Asignatura: Algebra y Trigonometria
Horas de trabajo docente: 96
Horas de trabajo independiente: 120
Número total de estudiantes: 30
Código del curso: AYT01301-Algebraytrigonometria-30-02
Total de cursos asignados: 5
Fecha de creación: 20240601

Código Asignatura: AYT01301
Nombre Asignatura: Algebra y Trigonometria
Horas de trabajo docente: 96
Horas de trabajo independiente: 120
Número total de estudiantes: 30
Código del curso: AYT01301-Algebraytrigonometria-30-03
Total de cursos asignados: 5
Fecha de creación: 20240601

Código Asignatura: AYT01301
Nombre Asignatura: Algebra y Trigonometria
Horas de trabajo docente: 96
Horas de trabajo inde

'/content/Ruta_Trabajo_Final.zip'